In [1]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
from getpass import getpass

In [1]:
token = getpass()

NameError: name 'getpass' is not defined

### Conectar con la Collection en Mongo

In [8]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

### Extrer datos de FourSquare:

In [9]:
def foursquare_places(venue,location,category = ""):
    
    """
    Search for UNLIMITED places in a 10km radius given a key_word
    query (venue), and returns a .json()
    
    """
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": venue,
        "ll": location,
        "radius":10000,
        "categories":category,
        "limit":50,
        "sort":"DISTANCE"
    }

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    return requests.get(url, params=params, headers=headers).json()


### Insertar los datos de FourSquare en la coleccion

In [10]:
def insert_parents_fourquare(response):
    
    """
    Inserts the results of a reques that ARE NOT in the 
    foursquare collection AND ARE NOT related to other object(childrens)
    """
    
    # 1. set conter for inserted objects
    inserted = 0
    
    # 2. access foursquare collection
    c = access_mdb_local_collection ("ironhack", "foursquare")
    
    # 3. Get the list of all FS elements alrready registered
    registered = [i['fsq_id'] for i in c.find({}, {"_id":0,"fsq_id":1})]
    
    # 4. Iterate trough all elements of the response 
    for i in response["results"]:
        
        #Need to meet 2 conditions to be inserted:
        # It is not alrready registered:
        cond1 = i['fsq_id'] not in registered
        
        ## It's not a children of a parent:
        cond2 = "parent" not in list(i["related_places"].keys())
        
        # Check if the conditions are meet:
        if cond1 and cond2 == True:
            c.insert_one(i)
            inserted += 1
            
    print(f'{inserted} items inserted into foursquare collection')
               
    return

# MAIN:

In [11]:
locations = {
    "Madrid" : "40.40841191,-3.68760088",
    "Barcelona" : "41.38424664,2.17634927",
    "Valencia" : "39.47534441,-0.37565717",
    "Sevilla" : "37.38620512,-5.99251368"   
}

categories = {
    "stadium" : 10051,
    "dog_park" : 16033,
    "vegan_rest" : 13377,
    "nigth_club" : 10032
}

for key, loc in locations.items():
    for key2, cat in categories.items():
        
        venue = ""
        location = loc
        category = cat
        
        print(f'----Location {key}, category {key2}----')
        
        response = foursquare_places(venue,location,category)
    
        insert_parents_fourquare(response)
              
        print(f'---------------------------------------')
        

----Location Madrid, category stadium----
44 items inserted into foursquare collection
---------------------------------------
----Location Madrid, category dog_park----
49 items inserted into foursquare collection
---------------------------------------
----Location Madrid, category vegan_rest----
46 items inserted into foursquare collection
---------------------------------------
----Location Madrid, category nigth_club----
47 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category stadium----
49 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category dog_park----
43 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category vegan_rest----
49 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category nigth_club----
48 items inserted into foursquare collectio

### Hacemos lo mismo pero quitado lo de distancia asi sera por relevancia:

In [12]:
def foursquare_places(venue,location,category = ""):
    
    """
    Search for UNLIMITED places in a 10km radius given a key_word
    query (venue), and returns a .json()
    
    """
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": venue,
        "ll": location,
        "radius":10000,
        "categories":category,
        "limit":50,
    }

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    return requests.get(url, params=params, headers=headers).json()


In [13]:
locations = {
    "Madrid" : "40.40841191,-3.68760088",
    "Barcelona" : "41.38424664,2.17634927",
    "Valencia" : "39.47534441,-0.37565717",
    "Sevilla" : "37.38620512,-5.99251368"   
}

categories = {
    "stadium" : 10051,
    "dog_park" : 16033,
    "vegan_rest" : 13377,
    "nigth_club" : 10032
}

for key, loc in locations.items():
    for key2, cat in categories.items():
        
        venue = ""
        location = loc
        category = cat
        
        print(f'----Location {key}, category {key2}----')
        
        response = foursquare_places(venue,location,category)
    
        insert_parents_fourquare(response)
              
        print(f'---------------------------------------')
        

----Location Madrid, category stadium----
15 items inserted into foursquare collection
---------------------------------------
----Location Madrid, category dog_park----
0 items inserted into foursquare collection
---------------------------------------
----Location Madrid, category vegan_rest----
7 items inserted into foursquare collection
---------------------------------------
----Location Madrid, category nigth_club----
6 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category stadium----
7 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category dog_park----
2 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category vegan_rest----
13 items inserted into foursquare collection
---------------------------------------
----Location Barcelona, category nigth_club----
13 items inserted into foursquare collection
---